<a href="https://colab.research.google.com/github/shivu2507/Stock-Price-Prediction-based-on-news/blob/main/Dow_jones_stock_price_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dow Jones Opening Stock Price Prediction

The model predicts the opening stock price of dow jones index using the headlines of the previous day published on the previous day by reating wrod embeddings using glove model and using 1 Dimensional convolution for text and LSTM for sequence processing.

# Importing necessary libraries and dataset

In [1]:
!wget http://nlp.stanford.edu/data/glove.840B.300d.zip

--2020-12-08 12:16:25--  http://nlp.stanford.edu/data/glove.840B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.840B.300d.zip [following]
--2020-12-08 12:16:25--  https://nlp.stanford.edu/data/glove.840B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip [following]
--2020-12-08 12:16:26--  http://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2176768927 (2.0G) [application/zip

In [3]:
!unzip glove.840B.300d.zip

Archive:  glove.840B.300d.zip
  inflating: glove.840B.300d.txt     


In [4]:
import nltk 
nltk.download('stopwords') #To ignore the stopwords such as a, an,the, he, she .etc,. 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import re
import tensorflow as tf
from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split
from sklearn.metrics import median_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse

%matplotlib inline

In [75]:
from keras.models import Sequential, Model
from keras import initializers
from keras.layers import Dense, Activation, Embedding, Convolution1D, MaxPooling1D, Input, BatchNormalization, Flatten, Reshape, Concatenate, Dropout
from keras.layers.recurrent import LSTM, GRU
from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam, SGD, RMSprop
from keras import regularizers
from keras.utils.vis_utils import plot_model

In [7]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [9]:
dowjones = pd.read_csv('/content/drive/MyDrive/dowjones-news-data/dowjones-news-data/DowJones.csv')
news = pd.read_csv('/content/drive/MyDrive/dowjones-news-data/dowjones-news-data/News.csv')

# Glimpse and Overview of Dataset

In [10]:
dowjones.isnull().sum()

Date         0
Open         0
High         0
Low          0
Close        0
Volume       0
Adj Close    0
dtype: int64

In [11]:
news.isnull().sum()

Date    0
News    0
dtype: int64

In [12]:
dowjones.head()

,Date,Open,High,Low,Close,Volume,Adj Close
0,2016-07-01,17924.240234,18002.380859,17916.910156,17949.369141,82160000,17949.369141
1,2016-06-30,17712.759766,17930.609375,17711.800781,17929.990234,133030000,17929.990234
2,2016-06-29,17456.019531,17704.509766,17456.019531,17694.679688,106380000,17694.679688
3,2016-06-28,17190.509766,17409.720703,17190.509766,17409.720703,112190000,17409.720703
4,2016-06-27,17355.210938,17355.210938,17063.080078,17140.240234,138740000,17140.240234


In [13]:
news.head()

,Date,News
0,2016-07-01,A 117-year-old woman in Mexico City finally re...
1,2016-07-01,IMF chief backs Athens as permanent Olympic host
2,2016-07-01,"The president of France says if Brexit won, so..."
3,2016-07-01,British Man Who Must Give Police 24 Hours' Not...
4,2016-07-01,100+ Nobel laureates urge Greenpeace to stop o...


In [14]:
dowjones.shape

(1989, 7)

In [15]:
news.shape

(73608, 2)

In [18]:
print("Number of days in stock price data : {}" .format(len(set(dowjones.Date))))
print("Number of days in news data : {}" .format(len(set(news.Date))))


Number of days in stock price data : 1989
Number of days in news data : 1989


# Data Preparation

## Step 1 : Keeping Common date rows

In [17]:
news = news[news.Date.isin(dowjones.Date)]  #Including only the dates that are present in the dowjones dataset.

print('Number of days in stock price data : {}'.format(len(set(dowjones.Date))))
print('Number of days in news data : {}'.format(len(set(news.Date))))


Number of days in stock price data : 1989
Number of days in news data : 1989


## Step 2 : Removing unnecessary columns

In [19]:
dowjones = dowjones.drop(['High','Low','Close','Volume','Adj Close'], axis = 1)
dowjones = dowjones.set_index('Date')
dowjones.head()

,Open
Date,
2016-07-01,17924.240234
2016-06-30,17712.759766
2016-06-29,17456.019531
2016-06-28,17190.509766
2016-06-27,17355.210938


## Step 3 : Creating Target Variable

In [20]:
# Target Variable, Diff = Tomorrow's Open Price - Today's Open Price
# Based on the previous day news we will predict by how much the open price will decrease or increase from previous day's open price

dowjones['Diff'] = -1 * dowjones.diff(periods = 1)
dowjones['Date'] = dowjones.index
dowjones = dowjones.reset_index(drop = True)
dowjones = dowjones[dowjones.Diff.notnull()] #Removing the null values

In [21]:
dowjones.drop('Open', axis =1, inplace = True)
dowjones.head()

,Diff,Date
1,211.480468,2016-06-30
2,256.740235,2016-06-29
3,265.509765,2016-06-28
4,-164.701172,2016-06-27
5,-591.419921,2016-06-24


## Step 4 : Combining headlines of a date into a single list

In [22]:
price = []  
headlines = []  

for row in dowjones.iterrows():

  daily_headlines = [] # To store headlines of single date as a list of headlines.
  date = row[1]['Date']
  price.append(row[1]['Diff'])
  for row in news[news.Date == date].iterrows():
    daily_headlines.append(row[1]['News'])
  
  headlines.append(daily_headlines)

  if len(price) % 500 == 0:
    print(len(price))

500
1000
1500


In [23]:
# Printing sample input & output

price[0], headlines[0]

(211.48046800000157,
 ['Jamaica proposes marijuana dispensers for tourists at airports following legalisation: The kiosks and desks would give people a license to purchase up to 2 ounces of the drug to use during their stay',
  "Stephen Hawking says pollution and 'stupidity' still biggest threats to mankind: we have certainly not become less greedy or less stupid in our treatment of the environment over the past decade",
  'Boris Johnson says he will not run for Tory party leadership',
  'Six gay men in Ivory Coast were abused and forced to flee their homes after they were pictured signing a condolence book for victims of the recent attack on a gay nightclub in Florida',
  'Switzerland denies citizenship to Muslim immigrant girls who refused to swim with boys: report',
  'Palestinian terrorist stabs israeli teen girl to death in her bedroom',
  'Puerto Rico will default on $1 billion of debt on Friday',
  'Republic of Ireland fans to be awarded medal for sportsmanship by Paris mayor.',

## Normalizing the Target Variable

In [24]:
max_price = max(price)
min_price = min(price)
mean_price = np.mean(price)
def normalize_price(price):
  return ((price - min_price)/(max_price - min_price))

In [26]:
norm_price = []
for p in price:
  norm_price.append(normalize_price(p))

In [27]:
print("Maximum Normalized Price : {}".format(max(norm_price)))
print("Minimum Normalized Price : {}".format(min(norm_price)))
print("Mean Normalized Price : {}".format(np.mean(norm_price)))

Maximum Normalized Price : 1.0
Minimum Normalized Price : 0.0
Mean Normalized Price : 0.4551577545098642


In [28]:
print(max(len(i) for i in headlines))
print(min(len(i) for i in headlines))
print(np.mean([len(i) for i in headlines]))

25
22
24.996478873239436


## Cleaning the headlines text

Expandng some of the words like 've to have 't to not and also removing unwanted characters and giving fullforms to abbreviations and ignoring stopwords.

In [29]:
# Function to expand usual short forms in english such as won't, can't, we're and so on.

def decontracted(phrase):
  if "'" in phrase:
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can't", "can not", phrase)

    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m"," am", phrase)

  return phrase

In [31]:
# Function to replace some of the text such as us, un & uk along with above mentioned and stopwords.

def clean_text(text):

  text =text.lower()

  if True:
    text = text.split()
    new_text = []

    for word in text:
      new_text.append(decontracted(word))

    text = " ".join(new_text)

  text = re.sub(r'&amp;', ' ', text)
  text = re.sub(r'0,0','00', text)
  text = re.sub(r'[_"\-;%()|.,+&=*%.,!?:#@\[\]]',' ', text)
  text = re.sub(r'\'', ' ', text)
  text = re.sub(r'\$','$', text)
  text = re.sub(r'u s','united states ', text)
  text = re.sub(r'u k','united kingdom ', text)
  text = re.sub(r'u n','united nations ', text)
  text = re.sub(r'j k',' jk ', text)
  text = re.sub(r' s ',' ', text)
  text = re.sub(r' yr ',' year ', text)
  text = re.sub(r' l g b t ','lgbt', text)
  text = re.sub(r'0km ','0 km ', text)
  

  text = text.split()
  stops = set(stopwords.words("english"))
  text = [w for w in text if not w in stops]
  text = " ".join(text)

  return text

In [37]:
# Calling above functions on the headlines.

clean_headlines = []

for daily_headlines in headlines:
  
  clean_daily_headlines = []
  
  for headline in  daily_headlines:
    clean_daily_headlines.append(clean_text(headline))
  
  clean_headlines.append(clean_daily_headlines)

In [38]:
clean_headlines[0]

['jamaica proposes marijuana dispensers tourists airports following legalisation kiosks desks would give people license purchase 2 ounces drug use stay',
 'stephen hawking says pollution istupidity still biggest threats mankind certainly become less greedy less stupid treatment environment past decade',
 'boris johnson says run tory party leadership',
 'six gay men ivory coast abused forced flee homes pictured signing condolence book victims recent attack gay nightclub florida',
 'switzerland denies citizenship muslim immigrant girls refused swim boys report',
 'palestinian terrorist stabs israeli teen girl death bedroom',
 'puerto rico default $1 billion debt friday',
 'republic ireland fans awarded medal sportsmanship paris mayor',
 'afghan suicide bomber kills 40 bbc news',
 'us airstrikes kill least 250 isis fighters convoy outside fallujah official says',
 'turkish cop took istanbul gunman hailed hero',
 'cannabis compounds could treat alzheimer removing plaque forming proteins br

In [53]:
words = [word for headlines in clean_headlines for headline in headlines for word in headline.split()]
print("Total number of unique words : {}".format(len(words)))

Total number of unique words : 615495


In [54]:
import collections

word_counts = collections.Counter(words)

## Creating embedding vector using GloVe(Global Vectors) model

In [55]:
embedding_index = {}

with open('/content/glove.840B.300d.txt', encoding = 'utf-8') as f:
  for line in f:
    values = line.split(' ')
    word = values[0]
    embedding = np.asarray(values[1:], dtype = 'float32')
    embedding_index[word] = embedding

print("Word Embedding : ",len(embedding_index))

Word Embedding :  2196016


In [47]:
# Since all words present in headlines won't be having the word embeddings we will define a threshold & remove words which occur below the threshold value.

threshold = 10

word_to_vector = {}

value = 0

for word, count in word_counts.items():
  if count >= threshold or word in embedding_index:
    word_to_vector[word] = value
    value += 1

In [48]:
len(word_to_vector)

31233

In [49]:
# Tokens for unknown words(UNK) and padding(PAD).

codes = ["<UNK>","<PAD>"]

for code in codes:
  word_to_vector[code] = len(word_to_vector)

# For reversing the vector to words
vector_to_word = {}
for word, value  in word_to_vector.items():
  vector_to_word[value] = word

usage_ratio = round(len(word_to_vector) / len(word_counts), 4)*100

print("Total Number of unique words : {}".format(len(word_counts)))
print("Number of words we will use : {}".format(len(word_to_vector)))
print("Percentage of word we will use : {}%".format(usage_ratio))

Total Number of unique words : 36710
Number of words we will use : 31235
Percentage of word we will use : 85.09%


## Creating embedding matrix

1. **Type 1 Words:**
<br>For words in headlines which occur above the threshold value and are present in GloVe we use glove word embeddings.<br>
2. **Type 2 Words:**
<br>For words which occur more than threshold value but not present in Glove, we will randomly initialise the word embeddings for those.
3. **Type 3 Words:**<br>For words which occur less than threshold and not present in GloVe, we will create a single token "<UNK>" for them and assign a single word embedding for them.

In [56]:
# Size of the vector dimensions

embedding_dim = 300

no_words = len(word_to_vector)

#Creating a zero matrix for Type 2 Words
word_embedding_matrix = np.zeros((no_words, embedding_dim))
for word, i in word_to_vector.items():
  if word in embedding_index:
    word_embedding_matrix[i] = embedding_index[word]
  else:
    new_embedding = np.array(np.random.uniform(-1.0, 1.0, embedding_dim))
    embedding_index[word] = new_embedding
    word_embedding_matrix[i] = new_embedding

print(len(word_embedding_matrix))

31235


In [58]:
word_count =0
unk_count = 0

headlines_sequence = []
for daily_headlines in clean_headlines:
  daily_headlines_seq = []
  for headlines in daily_headlines:
    headlines_seq = []
    for word in headlines.split():
      word_count +=1
      if word in word_to_vector:
        headlines_seq.append(word_to_vector[word])
      else:
        headlines_seq.append(word_to_vector["<UNK>"])
        unk_count += 1
    
    daily_headlines_seq.append(headlines_seq)
  headlines_sequence.append(daily_headlines_seq)
  
unk_percent = round(unk_count/word_count,4)*100

print("Total number of words in headlines : {}".format(word_count))
print("Total number of unknown words in headlines : {}".format(unk_count))
print("Total percent of unknown words in headlines : {}%".format(unk_percent))

Total number of words in headlines : 615495
Total number of unknown words in headlines : 8027
Total percent of unknown words in headlines : 1.3%


## Padding & Truncating

In [60]:
lengths = []
for headlines in headlines_sequence:
  for headline in headlines:
    lengths.append(len(headline))

lengths = pd.DataFrame(lengths, columns = ['Counts'])

In [61]:
lengths.describe()

,Counts
count,49693.000000
mean,12.385950
std,6.773807
min,1.000000
25%,7.000000
50%,10.000000
75%,16.000000
max,41.000000


### Observation

It can be seen that around 75% of the headlines are of length 16 words and to minimize training time we wil limit the max length of single headline to 16 words and that of a day to 200 words.

If a headline is less than 16 words we will keep it as it is, if it is longer than 16 words then we will truncate it from right and apend first 16 words.

If daily headlines is less than 200 words we will pad it, if it is greater than 200 words we will truncate it from the right side.

In [64]:
max_headline_length = 16
max_daily_length = 200
pad_headlines =[]

for headlines in headlines_sequence:
  pad_daily_headlines = []
  for headline in headlines:
    if len(headline) <= max_headline_length:
      for word in headline:
        pad_daily_headlines.append(word)
      
    else:
      headline = headline[:max_headline_length]
      for word in headline:
        pad_daily_headlines.append(word)

  if len(pad_daily_headlines) < max_daily_length:
    for i in range(max_daily_length - len(pad_daily_headlines)):
      pad = word_to_vector["<PAD>"]
      pad_daily_headlines.append(pad)

  else:
    pad_daily_headlines = pad_daily_headlines[:max_daily_length]

  pad_headlines.append(pad_daily_headlines)

# Split the Dataset

In [66]:
X_train, X_test, y_train, y_test = train_test_split(pad_headlines, norm_price, test_size = 0.15, random_state = 43)

X_train = np.array(X_train)
X_test = np.array(X_test)

y_train = np.array(y_train)
y_test = np.array(y_test)

In [68]:
print("Length of training datasets : X = {0} Y = {1}".format(len(X_train), len(y_train)))
print("Length of testing datasets : X = {0} Y = {1}".format(len(X_test), len(y_test)))

Length of training datasets : X = 1689 Y = 1689
Length of testing datasets : X = 299 Y = 299


# Model Building

## Hyperparameters

In [69]:
filter_length = 5
dropout = 0.5
learning_rate = 0.001
weights = initializers.TruncatedNormal(mean = 0.0, stddev = 0.1, seed = 2)
no_filter = 16
rnn_output_size = 128
hidden_dim = 128

## Creating model

In [85]:
def build_model():

  model = Sequential()

  # Creating an embedding layer.
  model.add(Embedding(no_words, embedding_dim, weights = [word_embedding_matrix], input_length = max_daily_length))
  model.add(Dropout(dropout))

  # Creating 2 1-Dimensional Convolutional layer.
  model.add(Convolution1D(filters = no_filter, kernel_size = filter_length, padding = 'same', activation = 'relu'))
  model.add(Dropout(dropout))

  model.add(Convolution1D(filters = no_filter, kernel_size = filter_length, padding = 'same', activation = 'relu'))
  model.add(Dropout(dropout))

  # Creating RNN Layer(LSTM).

  model.add(LSTM(rnn_output_size, activation=None, kernel_initializer=weights, dropout = dropout))

  # Creating Dense network

  model.add(Dense(hidden_dim, kernel_initializer=weights))
  model.add(Dropout(dropout))

  model.add(Dense(1, kernel_initializer=weights, name = 'output'))

  # Compiling the model.
  model.compile(loss = 'mean_squared_error', optimizer = Adam(lr = learning_rate, clipvalue = 1.0))#For solving exploding gradients problem clipvalue is assigned 1

  return model

## Fit the model

In [87]:
model = build_model()
print()
save_best_weights = 'best_weights.h5'

callbacks = [ModelCheckpoint(save_best_weights, monitor = 'val_loss', save_best_only = True),
             EarlyStopping(monitor = 'val_loss',patience = 5, verbose = 1, mode = 'auto'),
             ReduceLROnPlateau(monitor = 'val_loss', factor = 0.2, verbose = 1, patience = 3)]

history = model.fit([X_train], y_train, batch_size = 128, epochs = 50, validation_split = 0.15, verbose = True, shuffle = True,  callbacks = callbacks)

print(model.summary())


Epoch 1/50
12/12 [==============================] - 13s 1s/step - loss: 0.0594 - val_loss: 0.0455
Epoch 2/50
12/12 [==============================] - 12s 997ms/step - loss: 0.0252 - val_loss: 0.0443
Epoch 3/50
12/12 [==============================] - 12s 998ms/step - loss: 0.0195 - val_loss: 0.0369
Epoch 4/50
12/12 [==============================] - 12s 989ms/step - loss: 0.0153 - val_loss: 0.0279
Epoch 5/50
12/12 [==============================] - 12s 1s/step - loss: 0.0130 - val_loss: 0.0181
Epoch 6/50
12/12 [==============================] - 12s 1s/step - loss: 0.0111 - val_loss: 0.0122
Epoch 7/50
12/12 [==============================] - 12s 1s/step - loss: 0.0100 - val_loss: 0.0110
Epoch 8/50
12/12 [==============================] - 12s 1s/step - loss: 0.0106 - val_loss: 0.0119
Epoch 9/50
12/12 [==============================] - 12s 1s/step - loss: 0.0102 - val_loss: 0.0117
Epoch 10/50
12/12 [==============================] - 12s 999ms/step - loss: 0.0102 - val_loss: 0.0097
Epoch 

In [88]:
predictions = model.predict([X_test], verbose = True)

10/10 [==============================] - 1s 66ms/step


In [89]:
mse(y_test, predictions)

0.007373820714071627

In [97]:
def unnormalize(price):
  price = price*(max_price - min_price)+min_price
  return price

In [98]:
unnorm_predictions = []
for pred in predictions:
  unnorm_predictions.append(unnormalize(pred))

unnorm_y_test = []
for y in y_test:
  unnorm_y_test.append(unnormalize(y))

In [99]:
mse(unnorm_y_test, unnorm_predictions)

21174.920262106425

In [100]:
pd.Series(unnorm_y_test).describe()

count    299.000000
mean       6.710047
std      135.879128
min     -518.189453
25%      -51.799805
50%       11.708985
75%       81.265137
max      495.719727
dtype: float64

# Making predictions on news

In [101]:
def news_to_vector(news):
  vector = []
  for word in news.split():
    if word in word_to_vector:
      vector.append(word_to_vector[word])
    else:
      vector.append(word_to_vector["<UNK>"])
  
  return vector

In [103]:
def news_padding(news):

  padded_news = news
  if len(padded_news) < max_daily_length:
    for i in range(max_daily_length - len(padded_news)):
      padded_news.append(word_to_Vector["<PAD>"])

  elif len(padded_news)  > max_daily_length:
      padded_news = padded_news[:max_daily_length]

  return padded_news

In [106]:
todays_headlines = "Woman says note from Chinese 'prisoner' was hidden in new purse. \
               21,000 AT&T workers poised for Monday strike \
               housands march against Trump climate policies in D.C., across USA \
               Kentucky judge won't hear gay adoptions because it's not in the child's \"best interest\" \
               Multiple victims shot in UTC area apartment complex \
               Drones Lead Police to Illegal Dumping in Riverside County | NBC Southern California \
               An 86-year-old Californian woman has died trying to fight a man who was allegedly sexually assaulting her 61-year-old friend. \
               Fyre Festival Named in $5Million+ Lawsuit after Stranding Festival-Goers on Island with Little Food, No Security. \
               The \"Greatest Show on Earth\" folds its tent for good \
               U.S.-led fight on ISIS have killed 352 civilians: Pentagon \
               Woman offers undercover officer sex for $25 and some Chicken McNuggets \
               Ohio bridge refuses to fall down after three implosion attempts \
               Jersey Shore MIT grad dies in prank falling from library dome \
               New York graffiti artists claim McDonald's stole work for latest burger campaign \
               SpaceX to launch secretive satellite for U.S. intelligence agency \
               Severe Storms Leave a Trail of Death and Destruction Through the U.S. \
               Hamas thanks N. Korea for its support against ‘Israeli occupation’ \
               Baker Police officer arrested for allegedly covering up details in shots fired investigation \
               Miami doctor’s call to broker during baby’s delivery leads to $33.8 million judgment \
               Minnesota man gets 15 years for shooting 5 Black Lives Matter protesters \
               South Australian woman facing possible 25 years in Colombian prison for drug trafficking \
               The Latest: Deal reached on funding government through Sept. \
               Russia flaunts Arctic expansion with new military bases"

      
clean_news = clean_text(todays_headlines)

vector_news = news_to_vector(clean_news)

padded_news = news_padding(vector_news)

padded_news = np.array([padded_news]).reshape((1,-1))

pred = model.predict([padded_news])

price_change = unnormalize(pred)

print("The Dow Jones should open {} from the previous open".format(np.round(price_change[0][0],2)))

The Dow Jones should open -52.150001525878906 from the previous open
